# Notebook 06: Full Inference for Submission
**Mục tiêu:**
1. Chạy pipeline dự đoán trên **TẤT CẢ** các tập dữ liệu: Train, Validation, và Test.
2. Sinh ra file `pred.json` cho từng bài báo.
3. Sắp xếp kết quả vào đúng cấu trúc thư mục để nộp bài (Submission Ready).

**Model:** Decision Tree/XGBoost với **6 selected features**

**Cấu trúc Output:**
```bash
submission_final/
├── <Student_ID>/
│   ├── <paper_id_1>/
│   │   └── pred.json
│   ├── <paper_id_2>/
│   │   └── pred.json
...
```

In [17]:
import pandas as pd
import numpy as np
import json
import os
import sys
import joblib
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Setup path để import src module
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import từ src.ml module (Sử dụng lại các hàm đã refactor)
from src.ml import (
    load_json,
    save_json,
    normalize_text_basic,
    compute_pairwise_features,
    compute_tfidf_cosine_single,
    parse_bibtex_smart,
    transform_to_paper_based
)

pd.set_option('display.max_columns', None)
print("✅ Libraries và src.ml modules imported successfully!")

# --- DANH SÁCH 6 FEATURES ĐÃ CHỌN (phải khớp với Notebook 03/04) ---
SELECTED_FEATURES = [
    'feat_title_tfidf_cosine',
    'feat_title_len_diff',
    'feat_auth_jaccard',
    'feat_year_match',
    'feat_id_match',
    'feat_first_auth_match',
]
print(f"📝 Sử dụng {len(SELECTED_FEATURES)} selected features")

✅ Libraries và src.ml modules imported successfully!
📝 Sử dụng 6 selected features


In [18]:
# --- CẤU HÌNH QUAN TRỌNG ---

# 1. Thông tin sinh viên (Để tạo folder nộp bài)
STUDENT_ID = "23127011" 

# 2. Đường dẫn dữ liệu
DATASET_ROOT = '../../dataset_final'
PARTITIONS = ['train', 'validation', 'test']

# 3. Đường dẫn Model
MODEL_PATH = '../../dataset_final/models/best_matcher.pkl'
FEATURE_NAME_PATH = '../../dataset_final/models/feature_names.pkl'

# 4. Thư mục Output
SUBMISSION_DIR = f'submission_final/{STUDENT_ID}'

if not os.path.exists(SUBMISSION_DIR):
    os.makedirs(SUBMISSION_DIR)
    print(f"📁 Created submission directory: {os.path.abspath(SUBMISSION_DIR)}")
else:
    print(f"📁 Saving to existing directory: {os.path.abspath(SUBMISSION_DIR)}")


📁 Saving to existing directory: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_final\23127011


In [19]:
# --- OPTIMIZED HELPER FUNCTIONS WITH TIMING ---
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time

# Global timing stats
TIMING_STATS = {
    'feature_compute': 0.0,
    'tfidf_compute': 0.0,
    'model_predict': 0.0,
    'total_queries': 0
}

def batch_compute_features(pairs_list):
    """Compute features cho batch pairs - VECTORIZED."""
    if not pairs_list:
        return []
    
    results = []
    for row in pairs_list:
        feats = compute_pairwise_features(row)
        results.append(feats)
    return results


def rank_paper_fast(queries, candidates_list, model, feature_names):
    """
    Ranking tất cả queries của 1 paper cùng lúc - SUPER OPTIMIZED.
    
    KEY OPTIMIZATION: 
    - Pre-compute TF-IDF vectors cho candidates 1 LẦN
    - Reuse cho tất cả queries
    """
    global TIMING_STATS
    
    if not candidates_list:
        return {}
    
    predictions = {}
    
    # Pre-normalize titles 1 lần
    cand_titles = [normalize_text_basic(c.get('cand_title', '')) for c in candidates_list]
    cand_ids = [c['cand_id'] for c in candidates_list]
    query_titles = [normalize_text_basic(q.get('bib_title', '')) for q in queries]
    
    # === PRE-COMPUTE TF-IDF CHO TOÀN BỘ PAPER (1 LẦN DUY NHẤT) ===
    t_tfidf_start = time.time()
    
    # Gộp tất cả unique titles (queries + candidates)
    all_titles = list(set(query_titles + cand_titles))
    
    try:
        # Fit vectorizer 1 lần duy nhất cho paper này
        vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4), min_df=1)
        vectorizer.fit(all_titles)
        
        # Transform candidates 1 lần
        cand_vecs = vectorizer.transform(cand_titles)
        
        # Transform queries 1 lần  
        query_vecs = vectorizer.transform(query_titles)
        
        # Pre-compute tất cả TF-IDF scores (queries x candidates)
        # Shape: (n_queries, n_candidates)
        tfidf_matrix = cosine_similarity(query_vecs, cand_vecs)
        
    except Exception as e:
        # Fallback nếu lỗi
        tfidf_matrix = np.zeros((len(queries), len(candidates_list)))
    
    TIMING_STATS['tfidf_compute'] += time.time() - t_tfidf_start
    
    # === PROCESS TỪNG QUERY (đã có TF-IDF sẵn) ===
    for q_idx, query in enumerate(queries):
        bib_key = query['key']
        
        # Tạo pairs
        pairs = []
        for cand in candidates_list:
            row = {}
            row.update(query)
            row.update(cand)
            pairs.append(row)
        
        # --- PHASE 1: Feature Compute (KHÔNG có TF-IDF) ---
        t0 = time.time()
        feats_list = batch_compute_features(pairs)
        TIMING_STATS['feature_compute'] += time.time() - t0
        
        # Add TF-IDF từ pre-computed matrix
        for c_idx, feats in enumerate(feats_list):
            feats['feat_title_tfidf_cosine'] = tfidf_matrix[q_idx, c_idx]
        
        # Create DataFrame và predict
        df_feats = pd.DataFrame(feats_list)
        
        # Fill missing cols
        for col in feature_names:
            if col not in df_feats.columns:
                df_feats[col] = 0.0
        
        X_input = df_feats[feature_names].values
        
        # --- PHASE 2: Model Predict ---
        t2 = time.time()
        if model:
            scores = model.predict_proba(X_input)[:, 1]
        else:
            scores = np.random.rand(len(pairs))
        TIMING_STATS['model_predict'] += time.time() - t2
        
        # Ranking
        ranked_idx = np.argsort(scores)[::-1][:5]
        top_5 = [cand_ids[i] for i in ranked_idx]
        
        predictions[bib_key] = top_5
        TIMING_STATS['total_queries'] += 1
    
    return predictions


def print_timing_stats():
    """In thống kê thời gian của từng phase."""
    total = TIMING_STATS['feature_compute'] + TIMING_STATS['tfidf_compute'] + TIMING_STATS['model_predict']
    n_queries = max(TIMING_STATS['total_queries'], 1)
    
    print("\n" + "="*50)
    print("⏱️  TIMING BREAKDOWN")
    print("="*50)
    print(f"{'Phase':<25} | {'Total (s)':<12} | {'Avg/Query (ms)':<15} | {'%':<8}")
    print("-"*65)
    
    for phase, label in [
        ('feature_compute', 'Feature Extraction'),
        ('tfidf_compute', 'TF-IDF Cosine'),
        ('model_predict', 'Model Prediction')
    ]:
        t = TIMING_STATS[phase]
        pct = (t / total * 100) if total > 0 else 0
        avg_ms = (t / n_queries) * 1000
        print(f"{label:<25} | {t:>10.2f}s | {avg_ms:>12.2f}ms | {pct:>6.1f}%")
    
    print("-"*65)
    print(f"{'TOTAL':<25} | {total:>10.2f}s | {(total/n_queries)*1000:>12.2f}ms | 100.0%")
    print(f"\n📊 Processed {n_queries} queries")
    print("="*50)


def reset_timing_stats():
    """Reset timing stats."""
    global TIMING_STATS
    TIMING_STATS = {
        'feature_compute': 0.0,
        'tfidf_compute': 0.0,
        'model_predict': 0.0,
        'total_queries': 0
    }


print("✅ SUPER Optimized functions ready (TF-IDF pre-computed per paper).")

✅ SUPER Optimized functions ready (TF-IDF pre-computed per paper).


In [20]:
# --- LOAD MODEL ---
try:
    model = joblib.load(MODEL_PATH)
    feature_names = joblib.load(FEATURE_NAME_PATH)
    print(f"✅ Model loaded: {type(model).__name__}")
    print(f"📝 Expected features ({len(feature_names)}): {feature_names}")
    
    # Verify features match SELECTED_FEATURES
    if set(feature_names) == set(SELECTED_FEATURES):
        print("✅ Features khớp với SELECTED_FEATURES")
    else:
        print("⚠️ WARNING: Features không khớp! Sử dụng SELECTED_FEATURES")
        feature_names = SELECTED_FEATURES
        
except FileNotFoundError:
    print("❌ CRITICAL ERROR: Model file not found. Cannot proceed.")
    model = None
    feature_names = SELECTED_FEATURES

✅ Model loaded: Pipeline
📝 Expected features (6): ['feat_title_tfidf_cosine', 'feat_title_len_diff', 'feat_auth_jaccard', 'feat_year_match', 'feat_id_match', 'feat_first_auth_match']
✅ Features khớp với SELECTED_FEATURES



## Main Inference Loop
Vòng lặp này sẽ:
1. Duyệt qua từng Partition (Train -> Valid -> Test).
2. Load `labels.json` của partition đó.
3. Transform sang cấu trúc Paper-based.
4. Chạy Ranking Model.
5. Lưu file `pred.json` vào folder tương ứng với Paper ID.


In [21]:
# Biến để thống kê tổng kết
summary_stats = {
    'train': {'papers': 0, 'mrr': 0, 'queries': 0},
    'validation': {'papers': 0, 'mrr': 0, 'queries': 0},
    'test': {'papers': 0, 'mrr': 0, 'queries': 0}
}

# Reset timing trước khi chạy
reset_timing_stats()

print("🚀 STARTING OPTIMIZED INFERENCE PIPELINE...\n")

for partition in PARTITIONS:
    print(f"🔵 Processing Partition: [{partition.upper()}]")
    partition_start = time.time()
    
    # 1. Load Data
    t_load = time.time()
    label_file = os.path.join(DATASET_ROOT, partition, 'labels.json')
    if not os.path.exists(label_file):
        print(f"   ⚠️ Warning: File {label_file} not found. Skipping.")
        continue
        
    raw_data = load_json(label_file)
    if not raw_data:
        print("   ⚠️ Empty data. Skipping.")
        continue
    print(f"   ⏱️ Load data: {time.time() - t_load:.2f}s")

    # 2. Transform Data
    t_transform = time.time()
    papers_db = transform_to_paper_based(raw_data, parse_bibtex_smart)
    summary_stats[partition]['papers'] = len(papers_db)
    print(f"   ⏱️ Transform data: {time.time() - t_transform:.2f}s")
    
    print(f"   ↳ Found {len(papers_db)} papers. Ranking now...")
    
    # 3. OPTIMIZED Ranking Loop
    part_mrr_sum = 0
    part_query_count = 0
    
    for paper_id, data in tqdm(papers_db.items(), desc=f"   Ranking {partition}"):
        queries = data['queries']
        candidates_dict = data['candidates']
        candidates_list = list(candidates_dict.values())
        
        if not candidates_list or not queries:
            continue
        
        # --- FAST RANKING ---
        predictions = rank_paper_fast(queries, candidates_list, model, feature_names)
        
        # Build Output
        pred_output = {
            "partition": partition,
            "groundtruth": {q['key']: q['true_id'] for q in queries},
            "prediction": predictions
        }
        
        # MRR Calculation
        for query in queries:
            bib_key = query['key']
            true_id = query['true_id']
            top_5 = predictions.get(bib_key, [])
            
            if true_id in top_5:
                rank = top_5.index(true_id) + 1
                part_mrr_sum += 1.0 / rank
            part_query_count += 1
        
        # --- SAVE pred.json ---
        safe_paper_id = str(paper_id).strip()
        paper_dir = os.path.join(SUBMISSION_DIR, safe_paper_id)
        os.makedirs(paper_dir, exist_ok=True)
        
        save_path = os.path.join(paper_dir, 'pred.json')
        save_json(pred_output, save_path)
    
    # Update Stats
    if part_query_count > 0:
        summary_stats[partition]['mrr'] = part_mrr_sum / part_query_count
        summary_stats[partition]['queries'] = part_query_count
    
    print(f"   ✅ Finished {partition}. MRR: {summary_stats[partition]['mrr']:.4f}")
    print(f"   ⏱️ Partition total: {time.time() - partition_start:.2f}s")
    print("-" * 50)

# In timing breakdown sau khi chạy xong
print_timing_stats()

🚀 STARTING OPTIMIZED INFERENCE PIPELINE...

🔵 Processing Partition: [TRAIN]
   ⏱️ Load data: 0.25s
   ⏱️ Transform data: 290.45s
   ↳ Found 605 papers. Ranking now...


   Ranking train: 100%|██████████| 605/605 [39:16<00:00,  3.90s/it]  


   ✅ Finished train. MRR: 0.9973
   ⏱️ Partition total: 2647.71s
--------------------------------------------------
🔵 Processing Partition: [VALIDATION]
   ⏱️ Load data: 0.02s
   ⏱️ Transform data: 1.12s
   ↳ Found 1 papers. Ranking now...


   Ranking validation: 100%|██████████| 1/1 [00:07<00:00,  7.19s/it]


   ✅ Finished validation. MRR: 1.0000
   ⏱️ Partition total: 8.33s
--------------------------------------------------
🔵 Processing Partition: [TEST]
   ⏱️ Load data: 0.00s
   ⏱️ Transform data: 0.14s
   ↳ Found 1 papers. Ranking now...


   Ranking test: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]

   ✅ Finished test. MRR: 1.0000
   ⏱️ Partition total: 0.85s
--------------------------------------------------

⏱️  TIMING BREAKDOWN
Phase                     | Total (s)    | Avg/Query (ms)  | %       
-----------------------------------------------------------------
Feature Extraction        |    2310.78s |       171.91ms |   99.0%
TF-IDF Cosine             |      10.90s |         0.81ms |    0.5%
Model Prediction          |      12.61s |         0.94ms |    0.5%
-----------------------------------------------------------------
TOTAL                     |    2334.29s |       173.66ms | 100.0%

📊 Processed 13442 queries


In [22]:
# --- FINAL REPORT ---
print("\n" + "="*40)
print("📊 SUBMISSION GENERATION REPORT")
print("="*40)
print(f"{'Partition':<15} | {'Papers':<10} | {'Queries':<10} | {'MRR':<10}")
print("-" * 50)

for part in PARTITIONS:
    stats = summary_stats[part]
    print(f"{part.upper():<15} | {stats['papers']:<10} | {stats['queries']:<10} | {stats['mrr']:.4f}")

print("="*40)
print(f"\n📁 All pred.json files are ready at: {os.path.abspath(SUBMISSION_DIR)}")


📊 SUBMISSION GENERATION REPORT
Partition       | Papers     | Queries    | MRR       
--------------------------------------------------
TRAIN           | 605        | 13348      | 0.9973
VALIDATION      | 1          | 72         | 1.0000
TEST            | 1          | 22         | 1.0000

📁 All pred.json files are ready at: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_final\23127011
